In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

np.set_printoptions(precision=4, linewidth=200)

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
print(tf.__version__)

1.3.0


In [3]:
from utils.reader import ptb_raw_data
from utils.batcher import ptb_batcher
from utils.conditional_scope import cond_name_scope, cond_variable_scope
from utils.unrolled_rnn import make_rnn_variables
from utils.unrolled_rnn import make_rnn_outputs
from utils.unrolled_rnn import make_summary_nodes
from utils.unrolled_rnn import make_placeholders
from utils.unrolled_rnn import make_train_op
from utils.batcher import generate_epoch

In [4]:
X_train, X_val, X_test, vocab_size = ptb_raw_data('bigdata/simple-examples/data')

In [5]:
EMBEDDING_SIZE=64
HIDDEN_SIZE=256
BATCH_SIZE=32
NUM_STEPS=16
NUM_EPOCHS_INIT_LR=3
NUM_EPOCHS_TOTAL=8
INITIAL_LR=5e0
LR_DECAY_RATE=0.75
MAX_NORM=0.5

In [8]:
tf.reset_default_graph()
placeholders = make_placeholders(
    batch_size=BATCH_SIZE,
    num_steps=NUM_STEPS
)
rnn_vars = make_rnn_variables(
    vocab_size=vocab_size,
    embedding_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    initializer_scale=0.2
)
rnn_outputs = make_rnn_outputs(
    input_sequence=placeholders['inputs'],
    vocab_size=vocab_size,
    hidden_size=HIDDEN_SIZE,
    batch_size=BATCH_SIZE,
    num_steps=NUM_STEPS,
    rnn_variables=rnn_vars
)
summary_nodes = make_summary_nodes(
    targets=placeholders['targets'],
    logits=rnn_outputs['logits'],
)
training_nodes = make_train_op(
    summary_nodes['loss'],
    placeholders['learning_rate'],
    placeholders['max_norm'],
)

In [9]:
training_outputs = {**summary_nodes, **training_nodes}
with tf.Session() as sess:

    # Bookkeeping
    run_id = time.time()
    writer = tf.summary.FileWriter('logs/{0}'.format(run_id), sess.graph)
    coord = tf.train.Coordinator()
    sess.run(tf.global_variables_initializer())
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    learning_rate = INITIAL_LR
    max_norm = MAX_NORM
    for i in range(NUM_EPOCHS_TOTAL):
        if i >= NUM_EPOCHS_INIT_LR:
            learning_rate *= LR_DECAY_RATE
        for batch_idx, (inputs, targets) in enumerate(generate_epoch(X_train, BATCH_SIZE, NUM_STEPS)):
            outputs = sess.run(
                training_outputs,
                feed_dict={
                    placeholders['inputs']: inputs,
                    placeholders['targets']: targets,
                    placeholders['learning_rate']: learning_rate,
                    placeholders['max_norm']: max_norm,
                }
            )
            if (batch_idx % 64 == 63):
                print('step: {0}    loss: {1}    gradient norm: {2}     correct words: {3}'.format(
                    batch_idx+1,
                    outputs['loss'],
                    outputs['gradient_global_norm'],
                    outputs['num_correct_predictions'],
                ))
                
        total_loss, total_batches = 0, 0
        for inputs, targets in generate_epoch(X_val, BATCH_SIZE, NUM_STEPS):
            outputs = sess.run(
                summary_nodes,
                feed_dict={
                    placeholders['inputs']: inputs,
                    placeholders['targets']: targets
                },
            )
            total_loss += outputs['loss']
            total_batches += 1
        print('validation perplexity:', np.exp(total_loss / total_batches))
        total_loss, total_batches = 0, 0
        for inputs, targets in generate_epoch(X_test, BATCH_SIZE, NUM_STEPS):
            outputs = sess.run(
                summary_nodes,
                feed_dict={
                    placeholders['inputs']: inputs,
                    placeholders['targets']: targets
                },
            )
            total_loss += outputs['loss']
            total_batches += 1
        print('test perplexity:', np.exp(total_loss / total_batches))

    # Bookkeeping        
    writer.close()
    coord.request_stop()
    coord.join(threads)

step: 64    loss: 8.265625    gradient norm: 3.865234375     correct words: 37
step: 128    loss: 7.16015625    gradient norm: 0.7783203125     correct words: 46
step: 192    loss: 7.6953125    gradient norm: 3.443359375     correct words: 32
step: 256    loss: 6.875    gradient norm: 0.765625     correct words: 34
step: 320    loss: 6.3828125    gradient norm: 1.2119140625     correct words: 53
step: 384    loss: 6.34765625    gradient norm: 1.0400390625     correct words: 70
step: 448    loss: 6.36328125    gradient norm: 0.50537109375     correct words: 51
step: 512    loss: 6.7734375    gradient norm: 1.5263671875     correct words: 54
step: 576    loss: 6.4921875    gradient norm: 1.5439453125     correct words: 60
step: 640    loss: 6.0546875    gradient norm: 0.642578125     correct words: 63
step: 704    loss: 6.078125    gradient norm: 0.55126953125     correct words: 59
step: 768    loss: 6.2734375    gradient norm: 0.6455078125     correct words: 50
step: 832    loss: 6.0976

step: 960    loss: 4.7421875    gradient norm: 0.53515625     correct words: 116
step: 1024    loss: 5.27734375    gradient norm: 1.03125     correct words: 109
step: 1088    loss: 5.03125    gradient norm: 0.57666015625     correct words: 112
step: 1152    loss: 5.03515625    gradient norm: 0.54296875     correct words: 105
step: 1216    loss: 4.62109375    gradient norm: 0.58251953125     correct words: 124
step: 1280    loss: 4.7734375    gradient norm: 0.5849609375     correct words: 109
step: 1344    loss: 4.6796875    gradient norm: 0.5576171875     correct words: 114
step: 1408    loss: 4.7890625    gradient norm: 0.56298828125     correct words: 110
step: 1472    loss: 4.859375    gradient norm: 0.544921875     correct words: 123
step: 1536    loss: 4.9375    gradient norm: 0.89990234375     correct words: 106
step: 1600    loss: 4.828125    gradient norm: 0.52978515625     correct words: 124
step: 1664    loss: 4.8515625    gradient norm: 0.55859375     correct words: 118
step

step: 1792    loss: 4.4609375    gradient norm: 0.6064453125     correct words: 120
validation perplexity: 158.706612236
test perplexity: 149.551617614
step: 64    loss: 4.72265625    gradient norm: 0.56591796875     correct words: 115
step: 128    loss: 4.8671875    gradient norm: 0.6513671875     correct words: 115
step: 192    loss: 4.5078125    gradient norm: 0.58642578125     correct words: 118
step: 256    loss: 4.4296875    gradient norm: 0.58349609375     correct words: 119
step: 320    loss: 4.4296875    gradient norm: 0.578125     correct words: 138
step: 384    loss: 4.44921875    gradient norm: 0.54052734375     correct words: 135
step: 448    loss: 4.78125    gradient norm: 0.6318359375     correct words: 110
step: 512    loss: 4.609375    gradient norm: 0.5869140625     correct words: 115
step: 576    loss: 4.4609375    gradient norm: 0.5947265625     correct words: 134
step: 640    loss: 4.390625    gradient norm: 0.5791015625     correct words: 130
step: 704    loss: 4.